# CoMoGan
Notebook to test our model after training.

In [ ]:
import ipywidgets as widgets
import pytorch_lightning as pl
import pathlib
import torch
import yaml
import os

from math import pi
from PIL import Image
from munch import Munch
from threading import Timer
from IPython.display import clear_output
from torchvision.transforms import ToPILImage

from data import create_dataset
from torchvision.transforms import ToTensor
from data.base_dataset import get_transform
from networks import find_model_using_name, create_model

## Load the model with a checkpoint
Choose the directory that contains the checkpoint that you want.

In [ ]:
# Load names of directories inside /logs
p = pathlib.Path('./logs')
list_run_id = [str(x).split('/')[1] for x in p.iterdir() if x.is_dir()]

w_run = widgets.Dropdown(options=list_run_id,
                         description='Select RUN_ID',
                         disabled=False,
                         style=dict(description_width='initial'))

w_check = None
root_dir = None

def on_value_change_check(change):
    global w_check, w_run, root_dir
    
    clear_output(wait=True)
    
    root_dir = os.path.join('logs', w_run.value, 'tensorboard', 'default', 'version_0')
    p = pathlib.Path(root_dir + '/checkpoints')
    # Load a list of checkpoints, use the last one by default
    list_checkpoint = [str(x).split('checkpoints/')[1] for x in p.iterdir() if 'iter' in str(x)]
    list_checkpoint.sort(reverse = True, 
                         key=lambda x: int(x.split('_')[1].split('.pth')[0]))
    
    w_check = widgets.Dropdown(options=list_checkpoint,
                               description='Select checkpoint',
                               disabled=False,
                               style=dict(description_width='initial'))
    display(widgets.HBox([w_run, w_check]))

    
on_value_change_check({'new':w_run.value})
w_run.observe(on_value_change_check, names='value')

In [ ]:
RUN_ID = w_run.value
CHECKPOINT = w_check.value

# Load parameters
with open(os.path.join(root_dir, 'hparams.yaml')) as cfg_file:
    opt = Munch(yaml.safe_load(cfg_file))

opt.no_flip = True
# Load parameters to the model, load the checkpoint
model = create_model(opt)
model = model.load_from_checkpoint((os.path.join(root_dir, 'checkpoints/', CHECKPOINT)))
# Transfer the model to the GPU
model.to('cuda');

## Load the validation dataset

In [ ]:
# Switch train_only settings into validation
opt.dataroot = opt.dataroot.replace('train/', 'val/sunny/Day/')
# Load paths of all files contain in /Day
p = pathlib.Path(opt.dataroot)
dataset_paths = [str(x).replace(opt.dataroot,'') for x in p.iterdir()]
dataset_paths.sort()

sequence_name = {}
# Make a dict with each sequence name as a key and
# a list of paths to the images of the sequence as a value
for file in dataset_paths:
    # Keep only the sequence part contained in the name of the image
    strip_name = file.split('_')[0]
    
    if strip_name not in sequence_name:
        sequence_name[strip_name] = [file]
    else:
        sequence_name[strip_name].append(file)

## Select a sequence, an image and the moment of the day
Select the sequence on which you want to work before choosing which image should be used.<br>
Select the moment of the day, by choosing the angle of the sun $\phi$ between [0,2$\pi$],
which maps to a sun elevation ∈ [+30◦,−40◦]
<ul>
<li>0 means day</li>
<li>$\pi$/2 means dusk</li>
<li>$\pi$ means night</li>
<li>$\pi$ + $\pi$/2 means dawn</li>
<li>2$\pi$ means day (again)</li>
</ul>

In [ ]:
def drop_list():
    # Select the sequence on which you want to make your test
    return widgets.Dropdown(options=sequence_name.keys(),
                         description='Select sequence',
                         disabled=False,
                         style=dict(description_width='initial'))
def slider_img():
    # Select an image from the sequence
    return widgets.IntSlider(min=0, max=len(sequence_name[w_seq.value]) - 1,
                           description='Select image')
def slider_time():
    # Select time
    return widgets.FloatSlider(value=0, min=0, max=pi*2, step=0.01,
                             description='Select time',
                             readout_format='.2f')

def debounce(wait):
    # Decorator that will debounce the call to a function
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator
    
def inference(seq, index_img, phi, output = True):
    global sequence_name, w_img_time, w_seq, opt, out
    # Load the image
    A_path = os.path.join(opt.dataroot, sequence_name[seq.value][index_img])
    A_img = Image.open(A_path).convert('RGB')
    # Apply image transformation
    A = get_transform(opt, convert=False)(A_img)
    # Normalization between -1 and 1
    img_real = (((ToTensor()(A)) * 2) - 1).unsqueeze(0)
    # Forward our image into the model with the specified ɸ
    img_fake = model.forward(img_real.cuda(0), phi.cuda(0)) 
    # Encapsulate the initial image beside our result
    new_im = Image.new('RGB', (A_img.width * 2, A_img.height))
    new_im.paste(A_img, (0, 0))
    new_im.paste(ToPILImage()((img_fake[0].cpu() + 1) / 2), (A_img.width, 0))
    # Clear the output and display the widgets and the images
    if output:
        out.clear_output(wait=True)
        with out:
            # Resize the output
            O_img = new_im.resize((new_im.width // 2, new_im.height // 2))
            display(w_img_time, O_img)
        display(out)
    
    return new_im

@debounce(0.2)
def on_value_change_img(change):
    global w_seq, w_time
    inference(w_seq, change['new'], torch.tensor(w_time.value))
    
@debounce(0.2)
def on_value_change_time(change):
    global w_seq, w_img
    inference(w_seq, w_img.value, torch.tensor(change['new']))
    
def on_value_change_seq(change):
    global w_seq, w_img, w_time
    w_img = slider_img()
    w_time = slider_time()
    inference(w_seq, w_img.value, torch.tensor(w_time.value))
    
w_seq = drop_list()
w_img = slider_img()
w_time = slider_time()
w_img_time = widgets.VBox([w_seq, widgets.HBox([w_img, w_time])])
# Set the size of the output cell
out = widgets.Output(layout=widgets.Layout(width='auto', height='240px'))

inference(w_seq, w_img.value, torch.tensor(w_time.value))
w_img.observe(on_value_change_img, names='value')
w_time.observe(on_value_change_time, names='value')
w_seq.observe(on_value_change_seq, names='value')

## Sequence to video
The code below translates a sequence of images into a video.<br>
By default, the 'Select time' slider is on 'Variable phi' ($\phi$), in this case the time parameter will progressively increase from 0 to 2$\pi$.<br>
2$\pi$ is reached at the end of the video. If you move the slider, you can select a fixed $\phi$.<br>
Require to install two more packages <code>pip install imageio imageio-ffmpeg<code>

In [ ]:
from IPython.display import Video
import numpy as np
import imageio

w_button = widgets.Button(description='Start', tooltip='Start the inference of a sequence',
                          icon='play')

phi_opt = ['Variable phi'] + [str(round(f, 2)) for f in np.arange(0, pi*2, 0.01)]
w_vid_time = widgets.SelectionSlider(options=phi_opt, value=phi_opt[0], description='Select time : ')

w_vid_seq = drop_list()

w_button_seq = widgets.VBox([widgets.HBox([w_vid_seq, w_vid_time]), w_button])
display(w_button_seq)

def get_video(bt):
    global sequence_name, w_vid_seq, w_vid_time, w_button_seq
    
    clear_output(wait=True)
    display(w_button_seq)
    seq_size = len(sequence_name[w_vid_seq.value])
    # Display progress bar
    w_prog = widgets.IntProgress(value=0, min=0, max=seq_size, description='Loading:')
    display(w_prog)
    # Create a videos directory to save our video
    save_name = str(pathlib.Path('.').absolute()) + '/videos/'
    os.makedirs(save_name, exist_ok=True)
    # If variable phi
    if w_vid_time.value == 'Variable phi':
        # Write our video in the project folder
        save_name += 'comogan_{}_phi_{}.mp4'.format(w_vid_seq.value.replace('segment-', 'seg_'),
                                                   'variable')
    else:
        save_name += 'comogan_{}_phi_{}.mp4'.format(w_vid_seq.value.replace('segment-', 'seg_'),
                                                   w_vid_time.value.replace('.', '_'))
    writer = imageio.get_writer(save_name, fps=10)
    # Loop on the images contained in the sequence
    for i in range(seq_size):
        if w_vid_time.value == 'Variable phi':
            # Inference of the i image in sequence_name[key]
            phi_var = 2*pi/seq_size * i
            my_img = inference(w_vid_seq, i, torch.tensor(phi_var), False)
        else:
            my_img = inference(w_vid_seq, i, torch.tensor(float(w_vid_time.value)), False)
        writer.append_data(np.array(my_img))
        # Progress bar
        w_prog.value += 1
    
    writer.close()
    display(Video(save_name, embed=True))

w_button.on_click(get_video)